In [155]:
import somhos.resources.dataset as rd
import somhos.resources.queries as rq
import somhos.methods.useful as mu
import kleis.resources.dataset as kl
import hashlib as hl
import pickle
from pathlib import Path

In [2]:
kleis = kl.load_corpus()
kleis.training(features_method="simple-posseq", filter_min_count=10)

### [Aminer Citation Network Datasets - v1 and v9](https://aminer.org/citation)



In [135]:
#data_path = "resources/aminer/v1"
data_path = "../../src/somhos/resources/aminer/v9"
keyphrases_path = "../../src/somhos/resources/aminer/v9/kp-simple-posseq-10-nopostprocessing.txt"
keyphrases_count_path = "../../src/somhos/resources/aminer/v9/kps-count-simple-posseq10-nopostprocessing.pkl"
keyphrases_hash_path = "../../src/somhos/resources/aminer/v9/kps-hash-simple-posseq10-nopostprocessing.pkl"

In [4]:
related_docs = True
# Index 
ix_data = rq.cur_indexed_docs(data_path)
print(ix_data)

FileIndex(FileStorage('../../src/somhos/resources/aminer/v9/index-data'), 'MAIN')


In [143]:
if Path(keyphrases_hash_path).exists():
    with open(keyphrases_hash_path, "rb") as fin:
        kps_hashes_16 = pickle.load(fin)
else:
    kps_hashes_16 = {}

In [5]:
with ix_data.reader() as reader:
    i = 0
    kps_count = 0
    for doc_id, doc in reader.iter_docs():
        i += 1
        #if i < 900000:
        #    continue
        # if i >= 1000000:
        #    break
        try:
            keyphrases = kleis.label_text(doc['content'], post_processing=False)
            kps_count += len(keyphrases)
            for k in keyphrases:
                kps_hash_16 = mu.hash_16B(k[2])
        except ValueError:
            pass
    kps_count_hashes_16 = len(kps_hashes_16)
        # if len(kps) > 2:
    # print(kps_hashes)
    print(" - Documents: %d" % (i))
    print(" - Keyphrases: %d, %d, %d, %d" % (kps_count, kps_count_set, kps_count_hashes, kps_count_hashes_16))

 - Documents: 2385062
 - Keyphrases: 32839535, 27189590, 3761996, 3761996


In [154]:
len(kps_hashes_16)

3761996

In [142]:
if not Path(keyphrases_count_path).exists():
    kps_h = sorted(kps_hashes_16.items(), key=lambda x: x[1])
    print(len(kps_h), kps_h[:10])
    with open(keyphrases_count_path, "wb") as fout:
        pickle.dump([v for k,v in kps_h], fout, pickle.HIGHEST_PROTOCOL)
    with open(keyphrases_hash_path, "wb") as fout:
        pickle.dump({k: i for i, (k,v) in enumerate(kps_h)}, fout, pickle.HIGHEST_PROTOCOL)